In [1]:
#Se cargan las librerias y se crean los datos con los símbolos a eliminar
import unicodedata
from collections import Counter
from nltk.corpus import stopwords
import string
import re
import pandas as pd
import numpy as np
import plotly.express as px

_SYMBOLS = set(";:,.\\-\"'/()[]¿?¡!{}~<>|«»-—’\t\n\r")
quitar=string.punctuation
STOPWORDS_ES = stopwords.words("spanish")

In [2]:
#Normalize Text, eliminating multiple letters in words and accents
def normalize_text(text,
                   accents=False,
                   max_dup=2):

    nfkd_f = unicodedata.normalize('NFKD', text)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    return (unicodedata.normalize('NFKC', "".join(n_str)))

In [3]:
#This particular function eliminates the majory of the emojis
def deEmojify(x):
    regrex_pattern = re.compile(pattern= "["
                               u"\U00000000\U00000039"
                               u"\U00000100-\U000E0067"#u"\U00002660-\U0001F972"
                                   "]", flags=re.UNICODE)
    return regrex_pattern.sub(r'',x)

In [4]:
#Remover StopWords in spanish
def filter_words_es(text, remove_stopwords=True):
    wlist = text.split()
    o_text = []
    # filtrado de palabras
    for t in wlist:
        if remove_stopwords and t in STOPWORDS_ES:
            continue
        if t.isnumeric():
            continue
        o_text.append(t)
    return " ".join(o_text)

In [5]:
#This function applies all the functions created above
def limpieza(twitts):
    nuevo=[]
    for twitt in twitts:
        modificacion=re.sub(r'@\w+','', twitt) #elimina usuarios
        modificacion= re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', modificacion) #elimina URLs
        modificacion= re.sub(r'#\w+', '', modificacion) #elimina Hashtags
        modificacion=modificacion.lower() #cambia a minúsculas
        modificacion=normalize_text(modificacion) #normaliza el texto
        modificacion=filter_words_es(modificacion)
        for caracter in quitar:
            modificacion=modificacion.replace(caracter,
                         "")
            for caracter in _SYMBOLS:
                modificacion=modificacion.replace(caracter,
                             "")
        modificacion=deEmojify(modificacion) #elimina emoticones
        nuevo.append(modificacion)
    return nuevo

In [6]:
#This command load the file to clean
# "Path" is the rout were the document is found
Path="C:/Users/Luna/Documents/GitHub/MDScI/1. Data cleaning Techniques/..."
ruta = pd.read_json(Path,lines=True)
docs=limpieza(ruta["text"])

In [7]:
#Get all the words found in the JSON file and extracts the remaining word and count all the unique words
voc=[]
for d in docs:
    for t in d.split():
        voc.append(t.lower())
vocab=list(sorted(set(voc)))
print("tokens:",len(voc))
print("tokens únicos:", len(vocab))

tokens: 18546
tokens únicos: 6473
